## Burned area

In [17]:
workflow = dict([('id', 'burned-area'),
                ('label', 'Sentinel-2 burned area'),
                ('doc', 'Sentinel-2 burned area with NDVI/NDWI threshold')])


In [18]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')])

In [19]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')])

In [20]:
pre_event = dict([('id', 'pre_event'),
                  ('label', 'Sentinel-2 Level-2A pre-event'),
                  ('doc', 'Sentinel-2 Level-2A pre-event acquisition'),
                  ('value', '/workspace/ogc-tb16/wdir/5qoilug5'), 
                  ('type', 'Directory')])

In [21]:
post_event = dict([('id', 'post_event'),
                  ('label', 'Sentinel-2 Level-2A post-event'),
                  ('doc', 'Sentinel-2 Level-2A post-event acquisition'),
                  ('value', '/workspace/ogc-tb16/wdir/gclw8a9w/'), 
                  ('type', 'Directory')])

In [32]:
aoi = dict([('id', 'aoi'), 
              ('label', 'Area of interest'),
              ('doc', 'Area of interest in WKT'),
              ('value', 'POLYGON((136.508 -36.108,136.508 -35.654,137.178 -35.654,137.178 -36.108,136.508 -36.108))'), 
              ('type', 'string')])

In [34]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from time import sleep
from helpers import * 
from shapely.wkt import loads

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_burned_area/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
pre_s2 = os.path.join(pre_event['value'], 'catalog.json')
post_s2 = os.path.join(post_event['value'], 'catalog.json')

In [37]:
s2_items = dict()

s2_items['pre-event'] =  get_item(pre_s2)
s2_items['post-event'] = get_item(post_s2)

In [38]:
min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

for key, s2_item in s2_items.items():

    logging.info('Stacking bands for input {}'.format(key))
    vrt_bands = []

    for band in ['B04', 'B08', 'B11', 'SCL']:

        vrt_bands.append(s2_item.assets[band].get_absolute_href())

    vrt = '{}.vrt'.format(key)
    tif = '{}.tif'.format(key)

    logging.info('Build vrt for {}'.format(key))

    ds = gdal.BuildVRT(vrt,
                       vrt_bands,
                       srcNodata=0,
                       xRes=10, 
                       yRes=10,
                       separate=True)
    ds.FlushCache()


    logging.info('Translate {}'.format(key))

    gdal.Translate(tif,
                   vrt,
                   outputType=gdal.GDT_UInt16,
                  projWin=[min_lon, max_lat, max_lon, min_lat],
                  projWinSRS='EPSG:4326')
 
    os.remove(vrt)

In [39]:
ds = gdal.Open('pre-event.tif')

pre_b04 = ds.GetRasterBand(1).ReadAsArray()
pre_b08 = ds.GetRasterBand(2).ReadAsArray()
pre_b11 = ds.GetRasterBand(3).ReadAsArray()
pre_scl = ds.GetRasterBand(4).ReadAsArray()

ds = None

os.remove('pre-event.tif')

In [40]:
ds = gdal.Open('post-event.tif')

post_b04 = ds.GetRasterBand(1).ReadAsArray()
post_b08 = ds.GetRasterBand(2).ReadAsArray()
post_b11 = ds.GetRasterBand(3).ReadAsArray()
post_scl = ds.GetRasterBand(4).ReadAsArray()

width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

ds = None

os.remove('post-event.tif')

In [41]:
gain = 10000

pre_ndwi2 = (pre_b08 / gain - pre_b11 / gain) / (pre_b08 / gain  + pre_b11 / gain)
post_ndwi2 = (post_b08 / gain - post_b11 / gain) / (post_b08 / gain + post_b11 / gain)

pre_b11 = None
post_b11 = None

pre_ndvi = (pre_b08 / gain - pre_b04 / gain) / (pre_b08 / gain  + pre_b04 / gain)
post_ndvi = (post_b08 / gain - post_b04 / gain) / (post_b08 / gain + post_b04 / gain)

pre_b04 = None
post_b04 = None

pre_b08 = None
post_b08 = None

conditions = (((post_ndwi2 - pre_ndwi2) > float(ndwi_threshold['value'])) & ((post_ndvi - pre_ndvi) > float(ndvi_threshold['value'])) & (pre_scl == 4) | (post_scl == 4))  

burned = np.zeros((height, width), dtype=np.uint8) 

burned[conditions] = 1

pre_ndwi2 = None
post_ndwi2 = None

pre_ndvi = None
post_ndvi = None

burned[np.where((pre_scl == 0) | (post_scl == 0) | (pre_scl == 1) | (post_scl == 1) | (pre_scl == 5) | (post_scl == 5) | (pre_scl == 6) | (post_scl == 6) | (pre_scl == 7) | (post_scl == 7) | (pre_scl == 8) | (post_scl == 8) | (pre_scl == 9) | (post_scl == 9))] = 2 

In [42]:
output_name = 'S2_BURNED_AREA_{}'.format('_'.join([s2_item.datetime.strftime("%Y%m%d") for key, s2_item in s2_items.items()])) 

write_tif(burned, '{}.tif'.format(output_name),
          width,
          height,
          input_geotransform,
          input_georef)

In [43]:
catalog = Catalog(id='catalog', description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [44]:
result_titles = dict()

result_titles[output_name] = {'title': 'Burned area analysis from Sentinel-2',
                              'media_type': MediaType.COG}



items = []

for key, value in result_titles.items():

    result_item = Item(id=key,
                       geometry=s2_items['pre-event'].geometry,
                       bbox=s2_items['pre-event'].bbox,
                       datetime=s2_items['pre-event'].datetime,
                       properties={})

    result_item.add_asset(key='data',
                          asset=Asset(href='./{}.tif'.format(key), 
                          media_type=value['media_type'], 
                          title=value['title']))

    items.append(result_item)

#collection.add_items(items)

catalog.add_items(items)

In [45]:
catalog.describe()

catalog.normalize_and_save(root_href='./',
                           catalog_type=CatalogType.SELF_CONTAINED)

* <Catalog id=catalog>
  * <Item id=S2_BURNED_AREA_20200130_20201006>


In [46]:
shutil.move('{}.tif'.format(output_name), 
        os.path.join('./',
                     output_name,
                     '{}.tif'.format(output_name)))

'./S2_BURNED_AREA_20200130_20201006/S2_BURNED_AREA_20200130_20201006.tif'